In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import random
import torch
import time
import numpy as np
from gensim.models.word2vec import Word2Vec
from model import BatchProgramClassifier
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [23]:
root = '/home/david/projects/university/astnn/data/'
test_data = pd.read_pickle(root+'test/blocks.pkl')

word2vec = Word2Vec.load(root+"train/embedding/node_w2v_128").wv
embeddings = np.zeros((word2vec.vectors.shape[0] + 1, word2vec.vectors.shape[1]), dtype="float32")
embeddings[:word2vec.vectors.shape[0]] = word2vec.vectors

HIDDEN_DIM = 100
ENCODE_DIM = 128
LABELS = 104
EPOCHS = 15
BATCH_SIZE = 1
USE_GPU = False
MAX_TOKENS = word2vec.vectors.shape[0]
EMBEDDING_DIM = word2vec.vectors.shape[1]

model = BatchProgramClassifier(EMBEDDING_DIM,HIDDEN_DIM,MAX_TOKENS+1,ENCODE_DIM,LABELS,BATCH_SIZE,
                               USE_GPU, embeddings)
model.load_state_dict(torch.load("/home/david/projects/university/astnn/model.pt"))

<All keys matched successfully>

In [24]:
print(model)

BatchProgramClassifier(
  (encoder): BatchTreeEncoder(
    (embedding): Embedding(8189, 128)
    (W_c): Linear(in_features=128, out_features=128, bias=True)
    (W_l): Linear(in_features=128, out_features=128, bias=True)
    (W_r): Linear(in_features=128, out_features=128, bias=True)
  )
  (root2label): Linear(in_features=128, out_features=104, bias=True)
  (bigru): GRU(128, 100, batch_first=True, bidirectional=True)
  (hidden2label): Linear(in_features=200, out_features=104, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [55]:
len(test_data['code'][0])

29

In [56]:
test_data['code'][0]

[[32, [2, [30, [40, [4]]]]],
 [6],
 [2, [3055, [4]]],
 [2, [740, [4]]],
 [2, [740, [4]]],
 [2, [740, [4]]],
 [2, [257, [4]]],
 [2, [632, [4]]],
 [2, [835, [4]]],
 [59, [1150], [3055]],
 [3, [740], [57, [3055], [49]]],
 [3, [3055], [53, [3055], [49]]],
 [3, [740], [57, [3055], [127]]],
 [3, [3055], [53, [3055], [127]]],
 [3, [740], [57, [3055], [131]]],
 [3, [3055], [53, [3055], [131]]],
 [3, [257], [57, [3055], [60]]],
 [3, [3055], [53, [3055], [60]]],
 [3, [632], [57, [3055], [67]]],
 [3, [3055], [53, [3055], [67]]],
 [3, [835], [3055]],
 [28, [28, [51], [970]], [72]],
 [28, [28, [51], [1605]], [72]],
 [28, [28, [51], [1639]], [72]],
 [28, [28, [51], [722]], [72]],
 [28, [28, [51], [1539]], [72]],
 [28, [28, [51], [814]], [72]],
 [39, [5]],
 [7]]

In [54]:
embedder = model._modules['encoder']
embedder(test_data['code'][0], len(test_data['code'][0])).size()

torch.Size([29, 128])

In [50]:
model([test_data['code'][0]])

tensor([[  3.0438,  -9.1880,  -7.8805,  -7.5615,  -1.1363,  -1.5093,  -5.5260,
         -13.6943,  -1.4428,  -9.1244,   3.3101,  -2.1979,  -6.2667,  -5.7780,
          -4.6634,   8.9165,  -3.9500,  -5.2728,  -1.5660,  -3.7280,  -5.2731,
          -6.3635,  -0.6483,  -7.8033,   3.6247,  -3.6298,   1.8236,  -6.9257,
          -0.6329,   4.9817,   1.4290,  -4.7525,  -5.1086,   7.5247,  -8.0057,
           0.6907,  -6.3304,  -0.3814,  -1.9056,   1.2210,   1.9397,  -8.8574,
           0.1831,   4.7326,   0.3848,  -3.3588,  -2.2660,  -0.5018,  -6.9139,
           2.6575,  -6.2880,  -2.8472, -10.1830,   3.6429,  -0.0701,   6.9690,
          -0.2520,  -1.7929,  -6.3126,  -2.0208,   1.3635,  -3.7528,  -4.4504,
          -1.3831,  -4.9670,   6.9686,   2.6074,   1.5633,  -0.0940,  -7.4105,
          -1.8762,  -9.2367,  -6.4654,  -2.3879,  -7.8499,  -9.4226,  -2.8521,
           7.2459,  -2.4471,   4.3427,  -0.8219,  -6.4825,  -1.3890,  -6.3027,
          -4.2058,   1.8921,   2.2009,  -1.1523,  -5

## Get the gradient w.r.t to word embedding

In [15]:
test_data = pd.read_pickle('/home/david/projects/university/astnn/data/test/blocks.pkl')

In [49]:
# print(test_data['code'][0])
test_data.head([test_data['code'][0]])

[[32, [2, [30, [40, [4]]]]], [6], [2, [3055, [4]]], [2, [740, [4]]], [2, [740, [4]]], [2, [740, [4]]], [2, [257, [4]]], [2, [632, [4]]], [2, [835, [4]]], [59, [1150], [3055]], [3, [740], [57, [3055], [49]]], [3, [3055], [53, [3055], [49]]], [3, [740], [57, [3055], [127]]], [3, [3055], [53, [3055], [127]]], [3, [740], [57, [3055], [131]]], [3, [3055], [53, [3055], [131]]], [3, [257], [57, [3055], [60]]], [3, [3055], [53, [3055], [60]]], [3, [632], [57, [3055], [67]]], [3, [3055], [53, [3055], [67]]], [3, [835], [3055]], [28, [28, [51], [970]], [72]], [28, [28, [51], [1605]], [72]], [28, [28, [51], [1639]], [72]], [28, [28, [51], [722]], [72]], [28, [28, [51], [1539]], [72]], [28, [28, [51], [814]], [72]], [39, [5]], [7]]


TypeError: cannot do positional indexing on Int64Index with these indexers [[[[32, [2, [30, [40, [4]]]]], [6], [2, [3055, [4]]], [2, [740, [4]]], [2, [740, [4]]], [2, [740, [4]]], [2, [257, [4]]], [2, [632, [4]]], [2, [835, [4]]], [59, [1150], [3055]], [3, [740], [57, [3055], [49]]], [3, [3055], [53, [3055], [49]]], [3, [740], [57, [3055], [127]]], [3, [3055], [53, [3055], [127]]], [3, [740], [57, [3055], [131]]], [3, [3055], [53, [3055], [131]]], [3, [257], [57, [3055], [60]]], [3, [3055], [53, [3055], [60]]], [3, [632], [57, [3055], [67]]], [3, [3055], [53, [3055], [67]]], [3, [835], [3055]], [28, [28, [51], [970]], [72]], [28, [28, [51], [1605]], [72]], [28, [28, [51], [1639]], [72]], [28, [28, [51], [722]], [72]], [28, [28, [51], [1539]], [72]], [28, [28, [51], [814]], [72]], [39, [5]], [7]]]] of type list

In [42]:
import numpy as np 
import torch 
array = np.array(test_data['code'][0])
array
# output_data = torch.from_numpy(array)

<ipython-input-42-14bc3aa4690b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  array = np.array(test_data['code'][0])


array([list([32, [2, [30, [40, [4]]]]]), list([6]),
       list([2, [3055, [4]]]), list([2, [740, [4]]]),
       list([2, [740, [4]]]), list([2, [740, [4]]]),
       list([2, [257, [4]]]), list([2, [632, [4]]]),
       list([2, [835, [4]]]), list([59, [1150], [3055]]),
       list([3, [740], [57, [3055], [49]]]),
       list([3, [3055], [53, [3055], [49]]]),
       list([3, [740], [57, [3055], [127]]]),
       list([3, [3055], [53, [3055], [127]]]),
       list([3, [740], [57, [3055], [131]]]),
       list([3, [3055], [53, [3055], [131]]]),
       list([3, [257], [57, [3055], [60]]]),
       list([3, [3055], [53, [3055], [60]]]),
       list([3, [632], [57, [3055], [67]]]),
       list([3, [3055], [53, [3055], [67]]]), list([3, [835], [3055]]),
       list([28, [28, [51], [970]], [72]]),
       list([28, [28, [51], [1605]], [72]]),
       list([28, [28, [51], [1639]], [72]]),
       list([28, [28, [51], [722]], [72]]),
       list([28, [28, [51], [1539]], [72]]),
       list([28, [28, 

In [ ]:
loss = loss_function(output_data, Variable(train_labels))
loss.backward()

In [37]:
torch.autograd.Variable(torch.Tensor(10,10))

tensor([[ 1.2612e-44,  0.0000e+00,  2.4593e-37,  4.5785e-41,  1.3260e-37,
          0.0000e+00,         nan,         nan,  0.0000e+00,  0.0000e+00],
        [        nan,         nan, -8.0709e+31,  4.5783e-41,  1.4013e-45,
          4.0274e-11,  1.0089e-43,  0.0000e+00,  9.8091e-45,  0.0000e+00],
        [ 2.4593e-37,  4.5785e-41,  1.3260e-37,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.4013e-45,  0.0000e+00,  1.4013e-45,  0.0000e+00],
        [-8.0709e+31,  4.5783e-41,  1.4013e-45,  2.5609e-12,  2.0179e-43,
          0.0000e+00,  4.2039e-45,  0.0000e+00,  2.4593e-37,  4.5785e-41],
        [ 1.3260e-37,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.4013e-45,
          0.0000e+00,  1.4013e-45,  0.0000e+00,  1.4013e-44,  0.0000e+00],
        [ 1.4013e-45,  0.0000e+00,  3.0268e-43,  0.0000e+00,  1.2612e-44,
          0.0000e+00,  2.4593e-37,  4.5785e-41,  1.3260e-37,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  5.6052e-45,  0.0000e+00,  1.4013e-45,
          0.0000e+00,  0.0000e+0

reproduce prev results

# Training Accuracy

## No modifications

- Testing results(Acc): 0.9804826458994328

## Dead Code Additions

- 2 additions: Testing results(Acc): 0.9753869820209595
- 5 additions: Testing results(Acc): 0.9682722815113931


## Variable Renaming

- Testing results(Acc): 0.9478896259975003

## Combination

- Variable renaming, 5 dead code additions: Testing results(Acc): 0.9473127583886165
- Variable renaming, 10 dead code additions: Testing results(Acc): 0.946928179982694

# Efficiency Notes

## Variable renaming
- Test set (10401 code samples): ~1m 8s